In [31]:
import pandas as pd
import math
import pprint
import pydot

df=pd.read_csv('Dataset.csv')
df=df.iloc[:,1:]
print(df)
target=df.keys()[-1]

      Age  Income  Gender Marital Status Buys
0     <21    High    Male         Single   No
1     <21    High    Male        Married   No
2   21-35    High    Male         Single  Yes
3     >35  Medium    Male         Single  Yes
4     >35     Low  Female         Single  Yes
5     >35     Low  Female        Married   No
6   21-35     Low  Female        Married  Yes
7     <21  Medium    Male         Single   No
8     <21     Low  Female        Married  Yes
9     >35  Medium  Female         Single  Yes
10    <21  Medium  Female        Married  Yes
11  21-35  Medium    Male        Married  Yes
12  21-35    High  Female         Single  Yes
13    >35  Medium    Male        Married   No


In [23]:
def find_entropy(df):
  entropy=0.0
  den=df[target].count()
  for tval in df[target].unique():
    num=df[df[target]==tval][target].count()
    fraction=num/den
    entropy+=-fraction*math.log(fraction,2)
  return entropy

find_entropy(df)

0.9402859586706309

In [24]:
def find_att_entropy(df,attribute):
  entropy=0.0
  den=df[attribute].count()
  #print(den)
  for aval in df[attribute].unique():
    #print(aval)
    tempdf=df[df[attribute]==aval]
   # print(tempdf)
    ent=find_entropy(tempdf)
    num=tempdf[target].count()
    #print(num)
    entropy+=num/den*ent
  return entropy
    
print(find_att_entropy(df,'Age'))

0.6935361388961918


In [29]:
def find_win_att(df):
  IG=[]
  atts=df.keys()[:-1]
  for att in atts:
    IG.append((find_entropy(df)-find_att_entropy(df,att),att))
  IG.sort(reverse=True)
  return IG[0][1]
    
    
find_win_att(df)   

'Age'

In [35]:
def subtable(df,attribute,value):
  return df[df[attribute]==value]

def buildtree(df):
  tree={}
  tree[find_win_att(df)]={}
  
  for aval in df[find_win_att(df)].unique():
    temdf=subtable(df,find_win_att(df),aval)
    if len(temdf[target].unique())!=1:
      tree[find_win_att(df)][aval]=buildtree(temdf)
    else:
      tree[find_win_att(df)][aval]=temdf[target].unique()
  return tree

print(buildtree(df))
    

{'Age': {'<21': {'Gender': {'Male': array(['No'], dtype=object), 'Female': array(['Yes'], dtype=object)}}, '21-35': array(['Yes'], dtype=object), '>35': {'Marital Status': {'Single': array(['Yes'], dtype=object), 'Married': array(['No'], dtype=object)}}}}


In [38]:
def predict(data,tree):
  for key in tree.keys():
    for aval in tree[key]:
      if data[key]==aval:
        if isinstance(tree[key][aval],dict):
          return predict(data,tree[key][aval])
        else:
          return tree[key][aval]
        
data1={'Age':'<21','Gender':'Male','Marital Status':'Single'}
print(predict(data1,buildtree(df)))

['No']


In [0]:
def draw(parentname,childname):
  edge=pydot.Edge(parentname,childname)
  graph.add_edge(edge)
  
def visit(node,parent=None):
  for k,v in node.items():
    if isinstance(v,dict):
      if parent:
        draw(parent,k)
      visit(v,k)
    else:
      draw(parent,k)
      draw(k,k+'_'+v[0])
      
graph=pydot.Dot(graph_type='graph')
visit(buildtree(df))
graph.write_png('test.png')

  